In [1]:
# General imports 
import os 
import re
import numpy as np
import pandas as pd
from pathlib import Path
import scipy.stats as stats 
from skimage import measure
import matplotlib.pyplot as plt

# Pynwb imports
from hdmf_zarr import NWBZarrIO
from nwbwidgets import nwb2widget

In [2]:
# set data path
import sys
import platform
from os.path import join as pjoin

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_dir = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_dir = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_dir = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_dir = "/media/$USERNAME/Brain2025/"
    
print('data directory set to', data_dir)

data directory set to /data/


In [3]:
metadata = pd.read_csv(os.path.join(data_dir, 'bci_task_metadata', 'bci_metadata.csv'))

In [4]:
for subject_id in np.sort(metadata['subject_id'].unique()):
    this_mouse_metadata = metadata[metadata['subject_id'] == subject_id].sort_values(by='session_number')

In [5]:
this_mouse_metadata

,project_name,session_type,_id,name,subject_id,genotype,virus,date_of_birth,sex,modality,session_date,age,session_time,targeted_structure,ophys_fov,session_number
2,Brain Computer Interface,BCI single neuron stim,580448b2-4b93-4477-b6ac-6e2972792fe0,single-plane-ophys_772414_2025-01-27_15-57-05_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-01-27,104,14:23:26.693000,Primary Motor Cortex,FOV_01,5.0
25,Brain Computer Interface,BCI single neuron stim,f9228e1c-3132-4979-a358-4c53f0ce9c62,single-plane-ophys_772414_2025-02-04_13-21-29_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-02-04,112,13:21:29.644999,Primary Motor Cortex,FOV_02,11.0
3,Brain Computer Interface,BCI single neuron stim,c1be8449-7bef-4967-88f5-6a6ffc29e5a3,single-plane-ophys_772414_2025-02-06_16-47-44_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-02-06,114,16:47:44.152999,Primary Motor Cortex,FOV_02,13.0
4,Brain Computer Interface,BCI single neuron stim,15b395c0-1caf-4be4-a212-a13b9a4d1dfc,single-plane-ophys_772414_2025-02-10_11-15-26_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-02-10,118,11:15:26.609999,Primary Motor Cortex,FOV_03,14.0


In [68]:
# Get the directory for this dataset and load it
bci_data_dir = os.path.join(data_dir, 'brain-computer-interface')
print(bci_data_dir)
# Get the data folder for this session
session_dir = os.path.join(bci_data_dir, session_name)
print(session_dir)
# Now find the NWB file and set the path to load it
nwb_file = [file for file in os.listdir(session_dir) if 'nwb' in file][0]
nwb_path = os.path.join(session_dir, nwb_file)
print(nwb_path)
# Load the data
# # io = NWBZarrIO(nwb_path, mode = 'r') 
# # nwbfile_zarr = io.read()
with NWBZarrIO(str(nwb_path), 'r') as io:
    nwbfile = io.read()
    print('Loaded NWB file from:', nwb_path)

/data/brain-computer-interface
/data/brain-computer-interface/single-plane-ophys_772414_2025-02-10_11-15-26_processed_2025-08-04_23-06-21
/data/brain-computer-interface/single-plane-ophys_772414_2025-02-10_11-15-26_processed_2025-08-04_23-06-21/single-plane-ophys_772414_2025-02-10_11-15-26_behavior_nwb


/opt/conda/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.7.0 because version 2.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Loaded NWB file from: /data/brain-computer-interface/single-plane-ophys_772414_2025-02-10_11-15-26_processed_2025-08-04_23-06-21/single-plane-ophys_772414_2025-02-10_11-15-26_behavior_nwb


In [62]:
epoch_table = nwbfile.intervals["epochs"].to_dataframe()
epoch_table

,stimulus_name,start_frame,stop_frame,start_time,stop_time
id,,,,,
0,photostim,0,34651,0.000000,1154.475337
1,spont,34652,43019,1154.508654,1433.273918
2,BCI,43020,89945,1433.307235,2996.718253
3,spont_post,89946,95945,2996.751570,3196.621633
4,photostim_post,95946,131522,3196.654950,4381.948725


In [63]:
valid_mice = []

# Get all unique mouse IDs
for mouse_id in this_mouse_metadata['subject_id'].unique():
    
    # Get all sessions for this mouse
    mouse_sessions = this_mouse_metadata[this_mouse_metadata['subject_id'] == mouse_id]
    session_numbers = mouse_sessions['session_number'].values
    session_names = mouse_sessions['name'].values  # or your session folder name column

    session_results = []

    # Loop through each session
    for session_name, session_number in zip(session_names, session_numbers):
        epoch_path = f"/your/data/{mouse_id}/{session_name}/epoch_table.csv"
        
        try:
            epoch_table = pd.read_csv(epoch_table)
            is_valid = has_valid_photostim_structure(epoch_table)
        except Exception as e:
            print(f"Failed to load {epoch_path}: {e}")
            is_valid = False

        session_results.append((session_name, is_valid))

    valid_sessions = [s for s, ok in session_results if ok]

    if len(valid_sessions) > 0:
        valid_mice.append({
            'subject_id': mouse_id,
            'n_sessions': len(session_names),
            'valid_sessions': valid_sessions
        })


Failed to load /your/data/772414/single-plane-ophys_772414_2025-01-27_15-57-05_processed_2025-08-12_02-31-55/epoch_table.csv: argument of type 'method' is not iterable
Failed to load /your/data/772414/single-plane-ophys_772414_2025-02-04_13-21-29_processed_2025-08-12_06-14-42/epoch_table.csv: argument of type 'method' is not iterable
Failed to load /your/data/772414/single-plane-ophys_772414_2025-02-06_16-47-44_processed_2025-08-06_18-52-14/epoch_table.csv: argument of type 'method' is not iterable
Failed to load /your/data/772414/single-plane-ophys_772414_2025-02-10_11-15-26_processed_2025-08-04_23-06-21/epoch_table.csv: argument of type 'method' is not iterable
